In [ ]:
# Mistral-7B LoRA Fine-Tuning для SDGVM (Витте)
# =============================================
# Этот скрипт предназначен для Google Colab (бесплатный T4 GPU).
#
# ИНСТРУКЦИЯ:
# 1. Откройте Google Colab: https://colab.research.google.com
# 2. Создайте новый ноутбук
# 3. Выберите GPU: Runtime → Change runtime type → T4 GPU
# 4. Скопируйте этот код в ячейки и запустите по порядку
#
# Перед запуском загрузите файл witte_dataset.jsonl в Colab
# (через боковую панель Files → Upload)
















In [ ]:
# ============ ЯЧЕЙКА 1: Установка ============
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps trl peft accelerate bitsandbytes xformers

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-y64ukc5g/unsloth_67287244efd448b5a50b87d5a399632b
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-y64ukc5g/unsloth_67287244efd448b5a50b87d5a399632b
  Resolved https://github.com/unslothai/unsloth.git to commit 434b38f6e1d3b97f23d465bbdbefb53c1c835720
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 77.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.5/376.5 kB 42.2 MB/s eta 0:00:

In [ ]:
# ============ ЯЧЕЙКА 2: Загрузка модели ============
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    max_seq_length=2048,
    dtype=None,  # Автоопределение (bfloat16 на Ampere+, float16 на T4)
    load_in_4bit=True,
)

print("✅ Модель загружена")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.2.1: Fast Mistral patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.35. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

✅ Модель загружена


In [ ]:
import torch

if torch.cuda.is_available():
    print("✅ GPU доступен!")
    print(f"Имя GPU: {torch.cuda.get_device_name(0)}")
else:
    print("❌ GPU НЕдоступен. Убедитесь, что вы выбрали GPU в 'Среда выполнения' -> 'Сменить тип среды выполнения'.")

✅ GPU доступен!
Имя GPU: Tesla T4


In [ ]:
# ============ ЯЧЕЙКА 3: Добавление LoRA ============
model = FastLanguageModel.get_peft_model(
    model,
    r=16,                          # Ранг LoRA
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                     "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=42,
)

print("✅ LoRA адаптеры добавлены")
model.print_trainable_parameters()

Unsloth 2026.2.1 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


✅ LoRA адаптеры добавлены
trainable params: 41,943,040 || all params: 7,289,966,592 || trainable%: 0.5754


In [ ]:
# ============ ЯЧЕЙКА 4: Загрузка датасета ============
import json
from datasets import Dataset

# Загружаем JSONL файл
data = []
with open("witte_dataset.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        entry = json.loads(line.strip())
        data.append(entry)

print(f"✅ Загружено {len(data)} записей")

# Формат промпта для Mistral Instruct
def format_prompt(example):
    instruction = example.get("instruction", "")
    inp = example.get("input", "")
    output = example.get("output", "")

    if inp:
        text = f"""<s>[INST] {instruction}

{inp} [/INST] {output}</s>"""
    else:
        text = f"""<s>[INST] {instruction} [/INST] {output}</s>"""

    return {"text": text}

dataset = Dataset.from_list(data)
dataset = dataset.map(format_prompt)

print(f"✅ Датасет подготовлен")
print(f"Пример: {dataset[0]['text'][:300]}...")

✅ Загружено 3378 записей


Map:   0%|          | 0/3378 [00:00<?, ? examples/s]

✅ Датасет подготовлен
Пример: <s>[INST] Расскажи об этом периоде. [/INST] Но, конечно, все приезжающие (для этого случая была устроена громадная беседка для приезжающих) чувствовали и понимали, что произошло большое несчастье и находились под этим настроением. Подъехал и экипаж Ли-Хун-Чана с его свитой. Когда Ли-Хун-Чан вошел в ...


In [ ]:
# ============ ЯЧЕЙКА 5: Обучение ============
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=2048,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        num_train_epochs=3,           # 3 эпохи
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=42,
        output_dir="outputs",
        save_strategy="epoch",
    ),
)

print("🚀 Начинаем обучение...")
stats = trainer.train()
print(f"✅ Обучение завершено!")
print(f"   Время: {stats.metrics['train_runtime']:.0f} секунд")
print(f"   Loss: {stats.metrics['train_loss']:.4f}")

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/3378 [00:00<?, ? examples/s]

🚀 Начинаем обучение...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 3,378 | Num Epochs = 3 | Total steps = 1,269
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 7,289,966,592 (0.58% trained)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 1


wandb: You chose 'Create a W&B account'
wandb: Create an account here: https://wandb.ai/authorize?signup=true&ref=models
wandb: After creating your account, create a new API key and store it securely.
wandb: Paste your API key and hit enter:

...

wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: vaceslavsebanov (vaceslavsebanov-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Step,Training Loss
10,2.058800
20,1.808100
30,1.722700
40,1.758100
50,1.723500
60,1.706400
70,1.684300
80,1.662900
90,1.671800
100,1.636100


train/epoch,▁▁▂▂▂▂▂▂▃▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇██
train/grad_norm,▃▅▁▁▃▁▁▁▃▂▁▂▂▃▂▃▃▄▂▃▄▄▄▅▆▅▅▆▆▅█▇▆▇▆▇▆█▇▆
train/learning_rate,████▇▇▇▇▇▆▆▆▆▅▅▅▄▄▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁
train/loss,███▇▇▇▇▇▇▇▇▇▄▄▄▄▄▄▄▄▅▄▄▄▄▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁
total_flos,1.8422023207039795e+17
train/epoch,3
train/global_step,1269
train/grad_norm,1.26292
train/learning_rate,0.0
train/loss,0.9637


✅ Обучение завершено!
   Время: 9857 секунд
   Loss: 1.3097


In [ ]:
# ============ ЯЧЕЙКА 6: Тестирование ============
FastLanguageModel.for_inference(model)

test_prompts = [
    "Расскажи о золотом стандарте Витте.",
    "Как строился Транссиб?",
    "Какие отношения были у Витте с Николаем II?",
]

for prompt in test_prompts:
    inputs = tokenizer(f"<s>[INST] {prompt} [/INST]", return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=256, temperature=0.7)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"\n❓ {prompt}")
    print(f"💬 {response.split('[/INST]')[-1].strip()[:300]}")


❓ Расскажи о золотом стандарте Витте.
💬 Расскажи о золотом стандарте Витте.  Вследствие этого, когда я сделался министром финансов, то, конечно, первым делом я задумал установить золотой стандарт. Но установить его я не мог, потому что не было достаточного количества золота. В то время, как я был министром финансов, мы постоянно проводили

❓ Как строился Транссиб?
💬 Как строился Транссиб?  В это время я уже был начальником эксплоатации Юго-Западных железных дорог, а потому встречался с князем Барятинским, когда он приезжал на юго-западные дороги. Князь Барятинский был человек очень почтенный, очень воспитанный, но с большой долею фразистовки. Он был женат на кн

❓ Какие отношения были у Витте с Николаем II?
💬 Какие отношения были у Витте с Николаем II?  Вследствие этого, когда я был в Америке, я поехал к президенту Рузвельту и сказал ему, что я считаю, что если Россия и Америка будут поддерживать принцип неприкосновенности Китая, то Китай будет процветать, а если же они начнут вмешива

In [ ]:
# ============ ЯЧЕЙКА 7: Экспорт в GGUF ============
# GGUF формат нужен для LLMUnity
print("📦 Экспорт в GGUF (Q4_K_M)...")
model.save_pretrained_gguf(
    "mistral-witte",
    tokenizer,
    quantization_method="q4_k_m"  # ~4 ГБ файл
)

print("✅ Файл сохранён: mistral-witte-Q4_K_M.gguf")
print("📥 Скачайте его и положите в папку LLMUnity вашего проекта")

# ============ ЯЧЕЙКА 8: Скачивание ============
# Раскомментируйте для автоматического скачивания:
# from google.colab import files
# files.download("mistral-witte-unsloth.Q4_K_M.gguf")

print("\n🎉 ГОТОВО!")
print("Следующие шаги:")
print("1. Скачайте файл .gguf")
print("2. Положите в папку LLMUnity проекта")
print("3. В Unity: LLMCharacter → Model → выберите новый файл")

📦 Экспорт в GGUF (Q4_K_M)...
Unsloth: Merging model weights to 16-bit format...


config.json:   0%|          | 0.00/724 [00:00<?, ?B/s]

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Checking cache directory for required files...
Cache check failed: model-00001-of-00003.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


...

Unsloth: Preparing safetensor model files:  67%|██████▋   | 2/3 [02:35<01:23, 83.77s/it]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Unsloth: Merging weights into 16bit: 100%|██████████| 3/3 [04:37<00:00, 92.53s/it]


Unsloth: Merge process complete. Saved to `/content/mistral-witte`
Unsloth: Converting to GGUF format...
==((====))==  Unsloth: Conversion from HF to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF f16 might take 3 minutes.
\        /    [2] Converting GGUF f16 to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: Updating system package directories
Unsloth: All required system packages already installed!
Unsloth: Install llama.cpp and building - please wait 1 to 3 minutes
Unsloth: Cloning llama.cpp repository
Unsloth: Install GGUF and other packages
Unsloth: Successfully installed llama.cpp!
Unsloth: Preparing converter script...


Unsloth: [1] Converting model into f16 GGUF format.
This might take 3 minutes...
Unsloth: Initial conversion completed! Files: ['mistral-witte_gguf/mistral-7b-instruct-v0.3.F16.gguf']
Unsloth: [2] Converting GGUF f16 into q4_k_m. This might take 10 minutes...
Unsloth: Model files cleanup...
Unsloth: All GGUF conversions completed successfully!
Generated files: ['mistral-witte_gguf/mistral-7b-instruct-v0.3.Q4_K_M.gguf']
Unsloth: example usage for text only LLMs: llama.cpp/llama-cli --model mistral-witte_gguf/mistral-7b-instruct-v0.3.Q4_K_M.gguf -p "why is the sky blue?"
Unsloth: Saved Ollama Modelfile to mistral-witte_gguf/Modelfile
Unsloth: convert model to ollama format by running - ollama create model_name -f mistral-witte_gguf/Modelfile
✅ Файл сохранён: mistral-witte-Q4_K_M.gguf
📥 Скачайте его и положите в папку LLMUnity вашего проекта

🎉 ГОТОВО!
Следующие шаги:
1. Скачайте файл .gguf
2. Положите в папку LLMUnity проекта
3. В Unity: LLMCharacter → Model → выберите новый файл
